<a id='top'></a>

# Data Aggregation of StatsBomb Event data
##### Notebook to aggregated engineered [StatsBomb](https://statsbomb.com/)  Event data on a match-by-match basis


### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 31/12/2020<br>
Notebook last updated: 31/12/2020

![title](../../img/stats-bomb-logo.png)

Click [here](#section5) to jump straight to the Exploratory Data Analysis section and skip the [Task Brief](#section2), [Data Sources](#section3), and [Data Engineering](#section4) sections. Or click [here](#section6) to jump straight to the Conclusion.

___

<a id='sectionintro'></a>

## <a id='import_libraries'>Introduction</a>
This notebook aggregated previously engineered Event data from [StatsBomb](https://statsbomb.com/) on a match-by-match basis, manipulated as DataFrames using [pandas](http://pandas.pydata.org/) and [matplotlib](https://matplotlib.org/) for visualisation.

For more information about this notebook and the author, I'm available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/);
*    edd.j.webster@gmail.com;
*    [@eddwebster](https://www.twitter.com/eddwebster);
*    [linkedin.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/);
*    [github/eddwebster](https://github.com/eddwebster/);
*    [public.tableau.com/profile/edd.webster](https://public.tableau.com/profile/edd.webster);
*    [kaggle.com/eddwebster](https://www.kaggle.com/eddwebster); and
*    [hackerrank.com/eddwebster](https://www.hackerrank.com/eddwebster).

![title](../../img/fifa21eddwebsterbanner.png)

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/football_analytics) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/A%29%20Web%20Scraping/TransferMarkt%20Web%20Scraping%20and%20Parsing.ipynb).

___

<a id='sectioncontents'></a>

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Sources](#section3)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Data Dictionary](#section3.2)<br>
      3.    [Creating the DataFrame](#section3.3)<br>
      4.    [Initial Data Handling](#section3.4)<br>
      5.    [Export the Raw DataFrame](#section3.5)<br>         
4.    [Data Engineering](#section4)<br>
      1.    [Introduction](#section4.1)<br>
      2.    [Columns of Interest](#section4.2)<br>
      3.    [String Cleaning](#section4.3)<br>
      4.    [Converting Data Types](#section4.4)<br>
      5.    [Export the Engineered DataFrame](#section4.5)<br>
5.    [Exploratory Data Analysis (EDA)](#section5)<br>
      1.    [...](#section5.1)<br>
      2.    [...](#section5.2)<br>
      3.    [...](#section5.3)<br>
6.    [Summary](#section6)<br>
7.    [Next Steps](#section7)<br>
8.    [Bibliography](#section8)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>

This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing;
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation;
*    [`Beautifulsoup`](https://pypi.org/project/beautifulsoup4/) for web scraping; and
*    [`matplotlib`](https://matplotlib.org/contents.html?v=20200411155018) for data visualisations;

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [49]:
# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
import math
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd
import os
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os
from os.path import basename

# Flatten lists
from functools import reduce

# Working with JSON
import json
from pandas.io.json import json_normalize

# Web Scraping
import requests
from bs4 import BeautifulSoup
import re

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno

# Progress Bar
from tqdm import tqdm

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print('Setup Complete')

Setup Complete


In [50]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))
print('Seaborn: {}'.format(sns.__version__))

Python: 3.7.6
NumPy: 1.18.1
pandas: 1.0.1
matplotlib: 3.1.3
Seaborn: 0.10.0


### Defined Variables

In [51]:
# Define today's date
today = datetime.datetime.now().strftime('%d/%m/%Y').replace('/', '')

### Defined Filepaths

In [52]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..', )
data_dir = os.path.join(base_dir, 'data')
data_dir_sb = os.path.join(base_dir, 'data', 'sb')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

### Notebook Settings

In [53]:
pd.set_option('display.max_columns', None)

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>
This Jupyter notebook engineered data scraped from [StatsBomb](https://statsbomb.com/)  and FBref using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

The data of player values produced in this notebook is exported to CSV and used to create Age Profile visualisation in Tableau. See example [here](https://twitter.com/NdyStats/status/1343809317107986438).

---

<a id='section3'></a>

## <a id='#section3'>3. Data Sources</a>
Before conducting our EDA, the data needs to be imported as a DataFrame in the Data Sources section [Section 3](#section3) and cleaned in the Data Engineering section [Section 4](#section4).

We'll be using the [pandas](http://pandas.pydata.org/) library to import our data to this workbook as a DataFrame.

### <a id='#section3.2'>3.2. Data Dictionaries</a>
The [StatsBomb](https://statsbomb.com/)  dataset has six features (columns) with the following definitions and data types:

| Feature     | Data type    |
|------|-----|
| `position_number`    | object     |
| `position_description`    | object     |
| `name`    | object     |
| `dob`    | object     |
| `nationality`    | object     |
| `value`    | object     |

### <a id='#section3.3'>3.3. Import CSV as pandas DataFrame</a>

In [54]:
# Show files in directory
print(glob.glob(data_dir_sb + '/events/engineered/*.csv'))

['../../data/sb/events/engineered/sb_events_passing_network_1819_2021_wsl.csv', '../../data/sb/events/engineered/events_reading_stoke_07112020.csv', '../../data/sb/events/engineered/sb_events_1819_2021_wsl.csv']


In [55]:
# Import DataFrame as a CSV file
df_sb = pd.read_csv(data_dir_sb + '/events/engineered/sb_events_1819_2021_wsl.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
#df_sb = pd.read_csv(data_dir + '/export/sb_wsl_events.csv')

### <a id='#section3.4'>3.4. Preliminary Data Handling</a>
Let's quality of the dataset by looking first and last rows in pandas using the [head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [tail()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html) methods.

In [56]:
# Display the first 5 rows of the raw DataFrame, df_sb
df_sb.head()

id index period     timestamp minute  \
0  ac80414e-cec3-4c56-8e57-ac04149efbe2     5      1  00:00:01.324      0   
1  89cf3d24-ba04-4269-9071-1dfabf468cd1     6      1  00:00:02.553      0   
2  59972a9e-3362-43e1-96fd-883b2e3fbba4     7      1  00:00:02.553      0   
3  88d1ce50-3a00-4c88-90ba-a25e658c0deb     8      1  00:00:03.388      0   
4  9f0af77c-93cc-49d5-8d16-b7a640a0b59f     9      1  00:00:05.082      0   

  second possession  duration type.id      type.name possession_team.id  \
0      1          2  1.228695      30           Pass                971   
1      2          2       NaN      42  Ball Receipt*                971   
2      2          2  0.835305      43          Carry                971   
3      3          2  1.693583      30           Pass                971   
4      5          2       NaN      42  Ball Receipt*                971   

  possession_team.name play_pattern.id play_pattern.name team.id    team.name  \
0          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
1          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
2          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
3          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
4          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   

  tactics.formation tactics.lineup  \
0               NaN            NaN   
1               NaN            NaN   
2               NaN            NaN   
3               NaN            NaN   
4               NaN            NaN   

                                      related_events    location player.id  \
0           ['89cf3d24-ba04-4269-9071-1dfabf468cd1']  61.0, 41.0    4641.0   
1           ['ac80414e-cec3-4c56-8e57-ac04149efbe2']  52.0, 45.0   15549.0   
2  ['88d1ce50-3a00-4c88-90ba-a25e658c0deb', '89cf...  52.0, 45.0   15549.0   
3           ['9f0af77c-93cc-49d5-8d16-b7a640a0b59f']  53.0, 45.0   15549.0   
4           ['88d1ce50-3a00-4c88-90ba-a25e658c0deb']  38.0, 30.0    4633.0   

          player.name position.id             position.name pass.recipient.id  \
0     Francesca Kirby        23.0            Center Forward           15549.0   
1        Sophie Ingle         9.0  Right Defensive Midfield               NaN   
2        Sophie Ingle         9.0  Right Defensive Midfield               NaN   
3        Sophie Ingle         9.0  Right Defensive Midfield            4633.0   
4  Magdalena Ericsson         5.0          Left Center Back               NaN   

  pass.recipient.name pass.length  pass.angle pass.height.id pass.height.name  \
0        Sophie Ingle    9.848858   2.7233684            1.0      Ground Pass   
1                 NaN         NaN         NaN            NaN              NaN   
2                 NaN         NaN         NaN            NaN              NaN   
3  Magdalena Ericsson   21.213203  -2.3561945            1.0      Ground Pass   
4                 NaN         NaN         NaN            NaN              NaN   

  pass.end_location pass.type.id pass.type.name pass.body_part.id  \
0        52.0, 45.0         65.0       Kick Off              40.0   
1               NaN          NaN            NaN               NaN   
2               NaN          NaN            NaN               NaN   
3        38.0, 30.0          NaN            NaN              40.0   
4               NaN          NaN            NaN               NaN   

  pass.body_part.name carry.end_location under_pressure pass.outcome.id  \
0          Right Foot                NaN            NaN             NaN   
1                 NaN                NaN            NaN             NaN   
2                 NaN         53.0, 45.0            NaN             NaN   
3          Right Foot                NaN            NaN             NaN   
4                 NaN                NaN            NaN             NaN   

  pass.outcome.name ball_receipt.outcome.id ball_receipt.outcome.name  \
0               NaN      

In [57]:
# Display the last 5 rows of the raw DataFrame, df_sb
df_sb.tail()

id index period     timestamp  \
647276  36d07e5c-8608-4d1f-b090-4172c9f9bc01  3749      2  00:47:57.409   
647277  c9882596-3f63-4987-b97c-d25ef9458938  3750      2  00:48:00.720   
647278  1cfe4500-e93f-4fb6-a0ce-895dc0e8a835  3751      2  00:48:02.136   
647279  9275b5a5-e855-4072-a976-7459d92c40bc  3752      2  00:48:02.189   
647280  6c4fbaf2-47a3-4dd3-876e-eb43572fcc17  3753      2  00:48:02.189   

       minute second possession  duration type.id      type.name  \
647276     92     57        193  3.310232      43          Carry   
647277     93      0        193  1.416798      30           Pass   
647278     93      2        193       NaN      42  Ball Receipt*   
647279     93      2        193       0.0      34       Half End   
647280     93      2        193       0.0      34       Half End   

       possession_team.id possession_team.name play_pattern.id  \
647276                971          Chelsea FCW               1   
647277                971          Chelsea FCW               1   
647278                971          Chelsea FCW               1   
647279                971          Chelsea FCW               1   
647280                971          Chelsea FCW               1   

       play_pattern.name team.id            team.name tactics.formation  \
647276      Regular Play     971          Chelsea FCW               NaN   
647277      Regular Play     971          Chelsea FCW               NaN   
647278      Regular Play     971          Chelsea FCW               NaN   
647279      Regular Play     971          Chelsea FCW               NaN   
647280      Regular Play     969  Birmingham City WFC               NaN   

       tactics.lineup                                     related_events  \
647276            NaN  ['2f7add5d-4ce1-4504-a232-082ae203c738', 'c988...   
647277            NaN           ['1cfe4500-e93f-4fb6-a0ce-895dc0e8a835']   
647278            NaN           ['c9882596-3f63-4987-b97c-d25ef9458938']   
647279            NaN           ['6c4fbaf2-47a3-4dd3-876e-eb43572fcc17']   
647280            NaN           ['9275b5a5-e855-4072-a976-7459d92c40bc']   

          location player.id         player.name position.id  \
647276  22.0, 23.3    4633.0  Magdalena Ericsson         5.0   
647277  37.6, 24.0    4633.0  Magdalena Ericsson         5.0   
647278   64.2, 4.5   10222.0     Jonna Andersson         6.0   
647279         NaN       NaN                 NaN         NaN   
647280         NaN       NaN                 NaN         NaN   

           position.name pass.recipient.id pass.recipient.name pass.length  \
647276  Left Center Back               NaN                 NaN         NaN   
647277  Left Center Back           10222.0     Jonna Andersson   32.981964   
647278         Left Back               NaN                 NaN         NaN   
647279               NaN               NaN                 NaN         NaN   
647280               NaN               NaN                 NaN         NaN   

        pass.angle pass.height.id pass.height.name pass.end_location  \
647276         NaN            NaN              NaN               NaN   
647277  -0.6325858            1.0      Ground Pass         64.2, 4.5   
647278         NaN            NaN              NaN               NaN   
647279         NaN            NaN              NaN               NaN   
647280         NaN            NaN              NaN               NaN   

       pass.type.id pass.type.name pass.body_part.id pass.body_part.name  \
647276          NaN            NaN               NaN                 NaN   
647277          NaN            NaN              38.0           Left Foot   
647278          NaN            NaN               NaN                 NaN   
647279          NaN            NaN               NaN                 NaN   
647280          NaN            NaN               NaN                 NaN   

       carry.end_location under_pressure pass.outcome.id pass.outcome.name  \
647276         37.6, 24.0            NaN             Na

In [58]:
# Print the shape of the raw DataFrame, df_sb
print(df_sb.shape)

(647281, 207)


In [59]:
# Print the column names of the raw DataFrame, df_sb
print(df_sb.columns)

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'type.id', 'type.name',
       ...
       'location_y', 'pass.end_location_x', 'pass.end_location_y',
       'carry.end_location_x', 'carry.end_location_y', 'shot.end_location_x',
       'shot.end_location_y', 'shot.end_location_z',
       'goalkeeper.end_location_x', 'goalkeeper.end_location_y'],
      dtype='object', length=207)


The dataset has thirteen features (columns). Full details of these attributes can be found in the [Data Dictionary](section3.3.1).

In [60]:
# Data types of the features of the raw DataFrame, df_sb
df_sb.dtypes

id                           object
index                        object
period                       object
timestamp                    object
minute                       object
                              ...  
shot.end_location_x          object
shot.end_location_y          object
shot.end_location_z          object
goalkeeper.end_location_x    object
goalkeeper.end_location_y    object
Length: 207, dtype: object

All thirteen of the columns have the object data type. Full details of these attributes and their data types can be found in the [Data Dictionary](section3.3.1).

In [61]:
# Info for the raw DataFrame, df_sb
df_sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647281 entries, 0 to 647280
Columns: 207 entries, id to goalkeeper.end_location_y
dtypes: object(207)
memory usage: 1022.2+ MB


In [62]:
# Description of the raw DataFrame, df_sb, showing some summary statistics for each numberical column in the DataFrame
df_sb.describe()

id   index  period     timestamp  \
count                                 647281  647281  647281        647281   
unique                                647281    4379       2        412961   
top     1e554a2f-4fee-42e5-8876-3d5c66344163     580       1  00:00:00.000   
freq                                       1     194  331798            83   

        minute  second possession duration type.id type.name  \
count   647281  647281     647281   487510  647281    647281   
unique     103      60        254   328562      27        27   
top         45       1          2      0.0      30      Pass   
freq     15359   11356       4419    99515  176054    176054   

       possession_team.id possession_team.name play_pattern.id  \
count              647281               647281          647281   
unique                 13                   13               9   
top                   968          Arsenal WFC               1   
freq                80300                80300          287583   

       play_pattern.name team.id    team.name tactics.formation  \
count             647281  647281       647281               345   
unique                 9      13           13                22   
top         Regular Play     968  Arsenal WFC            4231.0   
freq              287583   74331        74331               123   

                                           tactics.lineup  \
count                                                 345   
unique                                                335   
top     [{'player': {'id': 22032, 'name': 'Hannah Hamp...   
freq                                                    3   

                                  related_events     location player.id  \
count                                     616014       644074    645788   
unique                                    590546       189496       314   
top     ['04b0f6fa-3de5-434e-8af2-b21670ed339a']  120.0, 80.0   10185.0   
freq                                           4          966      7765   

               player.name position.id     position.name pass.recipient.id  \
count               645788      645788            645788            159771   
unique                 314          25                25               314   
top     Stephanie Houghton         5.0  Left Center Back           10185.0   
freq                  7765       60379             60379              2141   

       pass.recipient.name pass.length pass.angle pass.height.id  \
count               159771      176054     176054         176054   
unique                 314       32303      53297              3   
top     Stephanie Houghton    9.219544        0.0            1.0   
freq                  2141         980       2966         102108   

       pass.height.name pass.end_location pass.type.id pass.type.name  \
count            176054            176054        42788          42788   
unique                3             82686            7              7   
top         Ground Pass         57.0, 4.0         66.0       Recovery   
freq             102108                46        19942          19942   

       pass.body_part.id pass.body_part.name carry.end_location  \
count             161448              161448             143026   
unique                 7                   7              69260   
top                 40.0          Right Foot         65.0, 77.0   
freq              106732              106732                 37   

       under_pressure pass.outcome.id pass.outcome.name  \
count          149153           47620             47620   
unique              1               5                 5   
top              True             9.0        Incomplete   
freq           149153           40191             40191   

       ball_receipt.outcome.id ball_receipt.outcome.name counterpress  \
count                    31337                     31337        22421   
unique                       1                         1            1   
top                        9.0  

In [63]:
# Plot visualisation of the missing values for each feature of the raw DataFrame, df_sb
msno.matrix(df_sb, figsize = (30, 7))

In [64]:
# Counts of missing values
sb_null_value_stats = df_sb.isnull().sum(axis=0)
sb_null_value_stats[sb_null_value_stats != 0]

duration                     159771
tactics.formation            646936
tactics.lineup               646936
related_events                31267
location                       3207
                              ...  
shot.end_location_x          642242
shot.end_location_y          642242
shot.end_location_z          643722
goalkeeper.end_location_x    644140
goalkeeper.end_location_y    644140
Length: 157, dtype: int64

The visualisation shows us very quickly that there a few missing values in most of the columns, but the dataset is ready for us.

## <a id='#section4'>4. Data Engineering</a>
Before we answer the questions in the brief through [Exploratory Data Analysis (EDA)](#section5), we'll first need to clean and wrangle the datasets to a form that meet our needs.

### <a id='#section4.1'>4.1. Assign DataFrames to Match-on_match DataFrame</a>

In [65]:
# Assign Raw DataFrames to new Engineered DataFrames
df_sb_mom = df_sb

### <a id='#section4.2'>4.2. Select Columns of Interest</a>

In [66]:
df_sb_mom.head()

id index period     timestamp minute  \
0  ac80414e-cec3-4c56-8e57-ac04149efbe2     5      1  00:00:01.324      0   
1  89cf3d24-ba04-4269-9071-1dfabf468cd1     6      1  00:00:02.553      0   
2  59972a9e-3362-43e1-96fd-883b2e3fbba4     7      1  00:00:02.553      0   
3  88d1ce50-3a00-4c88-90ba-a25e658c0deb     8      1  00:00:03.388      0   
4  9f0af77c-93cc-49d5-8d16-b7a640a0b59f     9      1  00:00:05.082      0   

  second possession  duration type.id      type.name possession_team.id  \
0      1          2  1.228695      30           Pass                971   
1      2          2       NaN      42  Ball Receipt*                971   
2      2          2  0.835305      43          Carry                971   
3      3          2  1.693583      30           Pass                971   
4      5          2       NaN      42  Ball Receipt*                971   

  possession_team.name play_pattern.id play_pattern.name team.id    team.name  \
0          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
1          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
2          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
3          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   
4          Chelsea FCW               9     From Kick Off     971  Chelsea FCW   

  tactics.formation tactics.lineup  \
0               NaN            NaN   
1               NaN            NaN   
2               NaN            NaN   
3               NaN            NaN   
4               NaN            NaN   

                                      related_events    location player.id  \
0           ['89cf3d24-ba04-4269-9071-1dfabf468cd1']  61.0, 41.0    4641.0   
1           ['ac80414e-cec3-4c56-8e57-ac04149efbe2']  52.0, 45.0   15549.0   
2  ['88d1ce50-3a00-4c88-90ba-a25e658c0deb', '89cf...  52.0, 45.0   15549.0   
3           ['9f0af77c-93cc-49d5-8d16-b7a640a0b59f']  53.0, 45.0   15549.0   
4           ['88d1ce50-3a00-4c88-90ba-a25e658c0deb']  38.0, 30.0    4633.0   

          player.name position.id             position.name pass.recipient.id  \
0     Francesca Kirby        23.0            Center Forward           15549.0   
1        Sophie Ingle         9.0  Right Defensive Midfield               NaN   
2        Sophie Ingle         9.0  Right Defensive Midfield               NaN   
3        Sophie Ingle         9.0  Right Defensive Midfield            4633.0   
4  Magdalena Ericsson         5.0          Left Center Back               NaN   

  pass.recipient.name pass.length  pass.angle pass.height.id pass.height.name  \
0        Sophie Ingle    9.848858   2.7233684            1.0      Ground Pass   
1                 NaN         NaN         NaN            NaN              NaN   
2                 NaN         NaN         NaN            NaN              NaN   
3  Magdalena Ericsson   21.213203  -2.3561945            1.0      Ground Pass   
4                 NaN         NaN         NaN            NaN              NaN   

  pass.end_location pass.type.id pass.type.name pass.body_part.id  \
0        52.0, 45.0         65.0       Kick Off              40.0   
1               NaN          NaN            NaN               NaN   
2               NaN          NaN            NaN               NaN   
3        38.0, 30.0          NaN            NaN              40.0   
4               NaN          NaN            NaN               NaN   

  pass.body_part.name carry.end_location under_pressure pass.outcome.id  \
0          Right Foot                NaN            NaN             NaN   
1                 NaN                NaN            NaN             NaN   
2                 NaN         53.0, 45.0            NaN             NaN   
3          Right Foot                NaN            NaN             NaN   
4                 NaN                NaN            NaN             NaN   

  pass.outcome.name ball_receipt.outcome.id ball_receipt.outcome.name  \
0               NaN      

In [67]:
# Select columns of interest

## Define columns
cols = ['match_date',
        'kick_off',
        'season_name',
        'competition_name',
        'Full_Fixture_Date',
        'match_week',
        'home_team.home_team_name',
        'away_team.away_team_name',
        'team.name',
        'type.name',
       ]

##
df_sb_mom_select = df_sb_mom[cols]

##
df_sb_mom_select['opponent.name'] = np.where(df_sb_mom_select['away_team.away_team_name'] == df_sb_mom_select['team.name'], df_sb_mom_select['home_team.home_team_name'], df_sb_mom_select['away_team.away_team_name'])


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [68]:
df_sb_mom_select

match_date      kick_off season_name         competition_name  \
0       2018-10-21  13:30:00.000   2018/2019  FA Women's Super League   
1       2018-10-21  13:30:00.000   2018/2019  FA Women's Super League   
2       2018-10-21  13:30:00.000   2018/2019  FA Women's Super League   
3       2018-10-21  13:30:00.000   2018/2019  FA Women's Super League   
4       2018-10-21  13:30:00.000   2018/2019  FA Women's Super League   
...            ...           ...         ...                      ...   
647276  2020-02-12  20:00:00.000   2019/2020  FA Women's Super League   
647277  2020-02-12  20:00:00.000   2019/2020  FA Women's Super League   
647278  2020-02-12  20:00:00.000   2019/2020  FA Women's Super League   
647279  2020-02-12  20:00:00.000   2019/2020  FA Women's Super League   
647280  2020-02-12  20:00:00.000   2019/2020  FA Women's Super League   

                                        Full_Fixture_Date match_week  \
0       2018-10-21 Birmingham City WFC 0  vs.  0 Chels...          6   
1       2018-10-21 Birmingham City WFC 0  vs.  0 Chels...          6   
2       2018-10-21 Birmingham City WFC 0  vs.  0 Chels...          6   
3       2018-10-21 Birmingham City WFC 0  vs.  0 Chels...          6   
4       2018-10-21 Birmingham City WFC 0  vs.  0 Chels...          6   
...                                                   ...        ...   
647276  2020-02-12 Chelsea FCW 2  vs.  0 Birmingham Ci...         16   
647277  2020-02-12 Chelsea FCW 2  vs.  0 Birmingham Ci...         16   
647278  2020-02-12 Chelsea FCW 2  vs.  0 Birmingham Ci...         16   
647279  2020-02-12 Chelsea FCW 2  vs.  0 Birmingham Ci...         16   
647280  2020-02-12 Chelsea FCW 2  vs.  0 Birmingham Ci...         16   

       home_team.home_team_name away_team.away_team_name            team.name  \
0           Birmingham City WFC              Chelsea FCW          Chelsea FCW   
1           Birmingham City WFC              Chelsea FCW          Chelsea FCW   
2           Birmingham City WFC              Chelsea FCW          Chelsea FCW   
3           Birmingham City WFC              Chelsea FCW          Chelsea FCW   
4           Birmingham City WFC              Chelsea FCW          Chelsea FCW   
...                         ...                      ...                  ...   
647276              Chelsea FCW      Birmingham City WFC          Chelsea FCW   
647277              Chelsea FCW      Birmingham City WFC          Chelsea FCW   
647278              Chelsea FCW      Birmingham City WFC          Chelsea FCW   
647279              Chelsea FCW      Birmingham City WFC          Chelsea FCW   
647280              Chelsea FCW      Birmingham City WFC  Birmingham City WFC   

            type.name        opponent.name  
0                Pass  Birmingham City WFC  
1       Ball Receipt*  Birmingham City WFC  
2               Carry  Birmingham City WFC  
3                Pass  Birmingham City WFC  
4       Ball Receipt*  Birmingham City WFC  
...               ...                  ...  
647276          Carry  Birmingham City WFC  
647277           Pass  Birmingham City WFC  
647278  Ball Receipt*  Birmingham City WFC  
647279       Half End  Birmingham City WFC  
647280       Half End          Chelsea FCW  

[647281 rows x 11 columns]

In [69]:
#

##
df_sb_mom_grouped = (df_sb_mom_select
                         #.groupby(['full_fixture_date'])
                          .groupby(['match_date',
                                    'kick_off',
                                    'season_name',
                                    'competition_name',
                                    'Full_Fixture_Date',
                                    'match_week',
                                    'home_team.home_team_name',
                                    'away_team.away_team_name',
                                    'team.name',
                                    'opponent.name',
                                    'type.name'])
                          .agg({'type.name': ['count']})
                     )

##
df_sb_mom_grouped.columns = df_sb_mom_grouped.columns.droplevel(level=0)


##
df_sb_mom_grouped = df_sb_mom_grouped.reset_index()

##
df_sb_mom_grouped = df_sb_mom_grouped.rename(columns={'match_date': 'match_date',
                                                      'kick_off': 'kick_off',
                                                      'season_name': 'season_name',
                                                      'competition_name': 'competition_name',
                                                      'Full_Fixture_Date': 'full_fixture_date',
                                                      'match_week': 'match_week',
                                                      'home_team.home_team_name': 'home_team',
                                                      'away_team.away_team_name': 'away_team',
                                                      'team.name': 'team_name',
                                                      'opponent.name': 'opponent_name',
                                                      'type.name': 'event_name',
                                                      'count': 'team_value'
                                                     }
                                            )


##
df_sb_mom_grouped.head()

match_date      kick_off season_name         competition_name  \
0  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
1  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
2  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
3  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
4  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   

                                full_fixture_date match_week    home_team  \
0  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
1  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
2  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
3  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
4  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   

       away_team    team_name  opponent_name     event_name  team_value  
0  Liverpool WFC  Arsenal WFC  Liverpool WFC  Ball Receipt*         590  
1  Liverpool WFC  Arsenal WFC  Liverpool WFC  Ball Recovery          61  
2  Liverpool WFC  Arsenal WFC  Liverpool WFC          Block          11  
3  Liverpool WFC  Arsenal WFC  Liverpool WFC          Carry         540  
4  Liverpool WFC  Arsenal WFC  Liverpool WFC      Clearance           4

In [70]:
# List unique values in the df_sb_mom_final['Event_Name'] column
df_sb_mom_grouped['event_name'].unique()

array(['Ball Receipt*', 'Ball Recovery', 'Block', 'Carry', 'Clearance',
       'Dispossessed', 'Dribble', 'Dribbled Past', 'Duel',
       'Foul Committed', 'Foul Won', 'Goal Keeper', 'Half End',
       'Injury Stoppage', 'Interception', 'Miscontrol', 'Pass',
       'Pressure', 'Shot', 'Substitution', 'Tactical Shift', 'Player Off',
       'Player On', 'Shield', 'Referee Ball-Drop', 'Error', 'Offside'],
      dtype=object)

In [71]:
# Select columns of interest

## Define columns
cols = ['full_fixture_date',
        'competition_name',
        'team_name',
        'opponent_name',
        'event_name',
        'team_value'
       ]

##
df_sb_mom_grouped_select = df_sb_mom_grouped[cols]

In [83]:
# Join DataFrame to itself on 'Date', 'Fixture', 'Team'/'Opponent', and 'Event', to join Team and Opponent together
df_sb_mom_final = pd.merge(df_sb_mom_grouped, df_sb_mom_grouped_select,  how='left', left_on=['full_fixture_date', 'competition_name', 'opponent_name', 'event_name'], right_on = ['full_fixture_date', 'competition_name', 'team_name', 'event_name'])


In [84]:
df_sb_mom_final.head()

match_date      kick_off season_name         competition_name  \
0  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
1  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
2  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
3  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
4  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   

                                full_fixture_date match_week    home_team  \
0  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
1  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
2  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
3  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
4  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   

       away_team  team_name_x opponent_name_x     event_name  team_value_x  \
0  Liverpool WFC  Arsenal WFC   Liverpool WFC  Ball Receipt*           590   
1  Liverpool WFC  Arsenal WFC   Liverpool WFC  Ball Recovery            61   
2  Liverpool WFC  Arsenal WFC   Liverpool WFC          Block            11   
3  Liverpool WFC  Arsenal WFC   Liverpool WFC          Carry           540   
4  Liverpool WFC  Arsenal WFC   Liverpool WFC      Clearance             4   

     team_name_y opponent_name_y  team_value_y  
0  Liverpool WFC     Arsenal WFC         303.0  
1  Liverpool WFC     Arsenal WFC          56.0  
2  Liverpool WFC     Arsenal WFC          28.0  
3  Liverpool WFC     Arsenal WFC         277.0  
4  Liverpool WFC     Arsenal WFC          20.0

In [85]:
# Clean Data

## Drop columns
df_sb_mom_final = df_sb_mom_final.drop(columns=['team_name_y', 'opponent_name_y'])

## Rename columns
df_sb_mom_final = df_sb_mom_final.rename(columns={'team_name_x': 'team_name',
                                                  'opponent_name_x': 'opponent_name',
                                                  'team_value_x': 'team_value',
                                                  'team_value_y': 'opponent_value',
                                                 }
                                        )

## Replace null values with zeros
df_sb_mom_final['team_value'] = df_sb_mom_final['team_value'].replace(np.nan, 0)
df_sb_mom_final['opponent_value'] = df_sb_mom_final['opponent_value'].replace(np.nan, 0)

## Convert Opponent_Value' from Float64 to Int64 type
df_sb_mom_final['opponent_value'] = df_sb_mom_final['opponent_value'].astype('Int64')

## Display DataFrame
df_sb_mom_final.head()

match_date      kick_off season_name         competition_name  \
0  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
1  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
2  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
3  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
4  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   

                                full_fixture_date match_week    home_team  \
0  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
1  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
2  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
3  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   
4  2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1  Arsenal WFC   

       away_team    team_name  opponent_name     event_name  team_value  \
0  Liverpool WFC  Arsenal WFC  Liverpool WFC  Ball Receipt*         590   
1  Liverpool WFC  Arsenal WFC  Liverpool WFC  Ball Recovery          61   
2  Liverpool WFC  Arsenal WFC  Liverpool WFC          Block          11   
3  Liverpool WFC  Arsenal WFC  Liverpool WFC          Carry         540   
4  Liverpool WFC  Arsenal WFC  Liverpool WFC      Clearance           4   

   opponent_value  
0             303  
1              56  
2              28  
3             277  
4              20

In [86]:
##
df_sb_mom_final_1819 = df_sb_mom_final[df_sb_mom_final['season_name'] == '2018/2019']
df_sb_mom_final_1920 = df_sb_mom_final[df_sb_mom_final['season_name'] == '2019/2020']

In [87]:
#

##
df_fixture_gw_1819 = (df_sb_mom_final_1819
                     .groupby(['full_fixture_date',
                               'team_name'
                              ])
                     .agg({'team_name': ['nunique']})
                )

##
df_fixture_gw_1819.columns = df_fixture_gw_1819.columns.droplevel(level=0)


##
df_fixture_gw_1819 = df_fixture_gw_1819.reset_index()

##
df_fixture_gw_1819 = df_fixture_gw_1819.rename(columns={'full_fixture_date': 'full_fixture_date',
                                              'nunique': 'game_week',
                                             }
                                    )

## Groupby. See: https://stackoverflow.com/questions/18554920/pandas-aggregate-count-distinct
df_fixture_gw_1819 = (df_fixture_gw_1819.groupby(['team_name', 'full_fixture_date']).sum()
                              .groupby(level=0).cumsum().reset_index()
                )

## Display DataFrame
df_fixture_gw_1819.head()

team_name                                  full_fixture_date  game_week
0  Arsenal WFC     2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1
1  Arsenal WFC   2018-09-19 Yeovil Town LFC 0  vs.  7 Arsenal WFC          2
2  Arsenal WFC  2018-09-23 Arsenal WFC 4  vs.  3 West Ham Unit...          3
3  Arsenal WFC       2018-10-14 Chelsea FCW 0  vs.  5 Arsenal WFC          4
4  Arsenal WFC       2018-10-21 Arsenal WFC 6  vs.  0 Reading WFC          5

In [88]:
#

##
df_fixture_gw_1920 = (df_sb_mom_final_1920
                     .groupby(['full_fixture_date',
                               'team_name'
                              ])
                     .agg({'team_name': ['nunique']})
                )

##
df_fixture_gw_1920.columns = df_fixture_gw_1920.columns.droplevel(level=0)


##
df_fixture_gw_1920 = df_fixture_gw_1920.reset_index()

##
df_fixture_gw_1920 = df_fixture_gw_1920.rename(columns={'full_fixture_date': 'full_fixture_date',
                                              'nunique': 'game_week',
                                             }
                                    )

## Groupby. See: https://stackoverflow.com/questions/18554920/pandas-aggregate-count-distinct
df_fixture_gw_1920 = (df_fixture_gw_1920.groupby(['team_name', 'full_fixture_date']).sum()
                                        .groupby(level=0).cumsum().reset_index()
                     )

## Display DataFrame
df_fixture_gw_1920.head()

team_name                                  full_fixture_date  game_week
0  Arsenal WFC  2019-09-08 Arsenal WFC 2  vs.  1 West Ham Unit...          1
1  Arsenal WFC  2019-09-16 Manchester United 0  vs.  1 Arsenal...          2
2  Arsenal WFC  2019-09-29 Arsenal WFC 4  vs.  0 Brighton & Ho...          3
3  Arsenal WFC       2019-10-13 Chelsea FCW 2  vs.  1 Arsenal WFC          4
4  Arsenal WFC  2019-10-27 Arsenal WFC 1  vs.  0 Manchester Ci...          5

In [89]:
df_fixture_gw = pd.concat([df_fixture_gw_1819, df_fixture_gw_1920])

In [90]:
# Join DataFrame 
df_sb_mom_final = pd.merge(df_sb_mom_final, df_fixture_gw, how='left', left_on=['full_fixture_date', 'team_name'], right_on = ['full_fixture_date', 'team_name'])

# Display DataFrame
df_sb_mom_final.head(50)

match_date      kick_off season_name         competition_name  \
0   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
1   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
2   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
3   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
4   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
5   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
6   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
7   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
8   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
9   2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
10  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
11  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
12  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
13  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
14  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
15  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
16  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
17  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
18  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
19  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
20  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
21  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
22  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
23  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
24  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
25  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
26  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
27  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
28  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
29  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
30  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
31  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
32  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
33  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
34  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
35  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
36  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
37  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
38  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
39  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
40  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
41  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
42  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
43  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
44  2018-09-09  13:30:00.000   2018/2019  FA Women's Super League   
45  2018-09-09  15:00:00.000   2018/2019  FA Women's Super League   
46  2018-09-09  15:00:00.000   2018/2019  FA Women's Super League   
47  2018-09-09  15:00:00.000   2018/2019  FA Women's Super League   
48  2018-09-09  15:00:00.000   2018/2019  FA Women's Super League   
49  2018-09-09  15:00:00.000   2018/2019  FA Women's Super League   

                                    full_fixture_date match_week  \
0      2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1   
1      2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1   
2      2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1   
3      2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1   
4      2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1   
5      2018-09-09 Arsenal WFC 5  vs.  0 Liverpool WFC          1   
6      2

### <a id='#section4.4'>4.4. Exporting the Engineered DataFrames</a>

In [91]:
df_sb_mom_final.to_csv(data_dir_sb + '/combined/engineered/csv/wsl/sb_wsl_events_mom_aggregated_1819_1920.csv', index=None, header=True)

df_sb_mom_final.to_csv(data_dir + '/export/sb_wsl_events_mom_aggregated.csv', index=None, header=True)

## <a id='#section5'>5. Summary</a>
This notebook aggregated previosuly engineered [StatsBomb](https://statsbomb.com/) Event data using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

## <a id='#section6'>6. Next Steps</a>
...

## <a id='#section7'>7. References</a>
...

---

***Visit my website [EddWebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)